#**XGBoost MISO For Linear Homogenous**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install xgboost


##Importing Libraries

In [ ]:
'''
from google.colab import files
uploaded = files.upload()
'''

import pandas as pd
import numpy as np

from google.colab import data_table
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from numpy import absolute
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import xgboost as xgb

In [ ]:
xgb.__version__

'1.4.2'

## Importing Datasets

here, the data is imported into dataframes - `df`

In [ ]:
#importing datasets
#Use workseet :alldata_allrows"

%reload_ext google.colab.data_table
df = pd.read_excel("/content/drive/MyDrive/00 MACHINE LEARNING/alldata_homogen_modev (1).xlsx",sheet_name="alldata_allrows")
#df = pd.read_excel("/content/drive/MyDrive/00 MACHINE LEARNING/alldata_homogen_modev.xlsx")

##Missing Data

NOTE:
  This code can only run once, else it registers error
  If you wish to run again, run the previous section first

`SECTION 1` identify any columns which has missing data

`SECTION 2` is where all columns contains NaN are replaced with median


In [ ]:
##############################################################################
#                       SECTION 1 : PRELIMINARY CHECK                        #
##############################################################################


nullver = df.isnull().values.any()
sumnull = df.isnull().sum().sum()

columns_with_nan = df.columns[nullver].tolist()
print("NaN Present: ", nullver)

#The same code is in the next section to double check if all NaN are replaced



##############################################################################
# SECTION 2: REPLACING MISSING DATA WITH MEDIAN VALUE OF A PARTICULAR COLUMN #
##############################################################################

'''
-  use: df['column name'] = df['column name'].replace(['old value'],'new value')

-  Replace using median :

      median = df_i['colname'].median()
      df['colname'].fillna(median, inplace=True)

      or

       df_i['colname'].fillna(df_i['colname'].median(),inplace=True)



'''

df = df.replace(['LSWR_CDU_KV100'],np.NaN)






NaN Present:  False


##Verify Missing Data is Fixed

In [ ]:
df.fillna(df.mean(),inplace=True)
nullver = df.isnull().values.any()
sumnull = df.isnull().sum().sum()

columns_with_nan = df.columns[nullver].tolist()
print("NaN Present: ", nullver)

df


## Train Test Split

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

#Use workseet :alldata_allrows", output in columns 22-27, inputs in columns 1-21. Minor changes from yesterday.
#meaning approx rows 1-1220 for training and model development, final test data rows 1221 to last row 1700++

X = df.iloc[:,0:21].values
y = df.iloc[:,21].values 

'''
X_train = df.iloc[0:1220, 0:21].values
X_test = df.iloc[1220:,0:21].values


#CHANGE HERE!!!
y_train = df.iloc[0:1220,26].values
y_test = df.iloc[1220:,26].values
'''


X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state = 42)


##Feature Scaling

In [ ]:

'''
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,:] = sc.fit_transform(X_train[:,:])
X_test[:,:] = sc.transform(X_test[:,:])
'''

'\nfrom sklearn.preprocessing import StandardScaler\nsc = StandardScaler()\nX_train[:,:] = sc.fit_transform(X_train[:,:])\nX_test[:,:] = sc.transform(X_test[:,:])\n'

##Developing Model

In [ ]:
'''

# regressor with hyperparameter tuning

RUN 1
Best Parameters: 
{'learning_rate': 1, 'max_depth': 2, 'n_estimators': 100, 'reg_lambda': 100}

Best Test Score: 
0.1918645064269362

RUN 2
Best Parameters: 
{'colsample_bynode': 0.5, 'colsample_bytree': 1, 'learning_rate': 1, 'max_depth': 2, 'n_estimators': 100, 'reg_lambda': 100}

Best Test Score: 
0.1959296488122364

RUN 3
Best Parameters: 
{'Max_delta_step': 0, 'gamma': 0, 'learning_rate': 1, 'max_depth': 2, 'n_estimators': 100, 'reg_lambda': 100}

Best Test Score: 
0.1918645064269362

RUN 4
Best Parameters: 
{'learning_rate': 1, 'max_depth': 2, 'n_estimators': 100, 'reg_lambda': 100}

Best Test Score: 
0.1918645064269362

RUn 4
Best Parameters: 
{'learning_rate': 1, 'max_depth': 2, 'n_estimators': 100, 'reg_lambda': 100}

Best Test Score: 
0.1918645064269362


'''

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

regressor = XGBRegressor()

params = {
    'n_estimators' : [100],
    'max_depth': [2],
    'learning_rate': [1], #eta
    'reg_lambda': [100],
}

grid_regressor = GridSearchCV(estimator = regressor,
                        param_grid = params,
                        cv = 5, 
                        verbose = 1,
                        n_jobs = -1)
grid_regressor.fit(X_train,y_train)


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.2s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estima...
                                    objective='reg:squarederror',
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
         

In [ ]:

print("Best Estimator: \n{}\n".format(grid_regressor.best_estimator_))
print("Best Parameters: \n{}\n".format(grid_regressor.best_params_))
print("Best Test Score: \n{}\n".format(grid_regressor.best_score_))
print("All Test Scores: \n{}\n".format(grid_regressor.cv_results_['mean_test_score']))

cv_result = pd.DataFrame(grid_regressor.cv_results_)


Best Estimator: 
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=1, max_delta_step=0, max_depth=2, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=2,
             num_parallel_tree=1, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=100, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

Best Parameters: 
{'learning_rate': 1, 'max_depth': 2, 'n_estimators': 100, 'reg_lambda': 100}

Best Test Score: 
0.1918645064269362

All Test Scores: 
[0.19186451 0.15609239]



## Training XGBoost on Training Set

To train:

`machine_learning_model.fit(input_training_Data, output_training_data)`

To test:

`output_prediction = machine_learning_model.predict(input_test_data)`

In [ ]:
#multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:linear')).fit(X, y)
regressor = XGBRegressor(max_depth = 2,n_estimators = 100,learning_rate=1,reg_lambda= 100)
regressor.fit(X_train, y_train)




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=1, max_delta_step=0, max_depth=2, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=2,
             num_parallel_tree=1, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=100, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Predict Test Result

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:

r2_score(y_test,y_pred)


mse = mean_squared_error(y_test, y_pred, sample_weight=None,squared=True)
rmse = mean_squared_error(y_test, y_pred, sample_weight=None,squared=False)
r2= r2_score(y_test, y_pred,sample_weight=None)


print('MSE Value: {:.5f}'.format(mse))
print('RMSE Value: {:.5f}'.format(rmse))
print('R-squared Value: {:.2f}'.format(r2))

MSE Value: 82.27240
RMSE Value: 9.07041
R-squared Value: 0.12


##Plotting Predicted outpus vs. Actual Data (Test Set)

Uncomment if you wish to plot Test Data vs. Predicted Data

In [ ]:
#print ypred and ytest in a single plot

import matplotlib.pyplot as plt
import numpy as np

#(row,co)
X_train.shape
X_test.shape
y_test.shape


plt.plot(y_test,'#0474BE', y_pred,'#D95319')
plt.legend(('Actual Data','Predicted Data'),
           loc='lower right', shadow=True )
plt.title('Tuned LSWR_Combine_Sim5 (TrainTestSplit)')
plt.savefig('Tuned LSWR_Combine_Sim5 ',dpi=300)
plt.show()

##Save Model

In [ ]:
from xgboost import Booster

regressor.save_model('Tuned LSWR_Combine_Sim5.model')


#bst.save_model('LSWR trained.model')



In [ ]:
import pickle

filename = '/content/drive/MyDrive/00 MACHINE LEARNING/Linear Homogenous/XGBoost/MISO/Tuned/Tuned LSWR_Combine_Dens15.model'
pickle.dump(regressor, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model



[01:37:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
loaded_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

result = loaded_model.score(X_test, y_test)

print(result)

Ypredict = loaded_model.predict(X_test)  

print("Predicted Dens15: \n ", Ypredict.reshape(521,1))

[14:13:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
-0.710012676396357
Predicted Dens15: 
  [[0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.879659  ]
 [0.8797605 ]
 [0.879659  ]
 [0.88118273]
 [0.88118273]
 [0.88118273]
 [0.88118273]
 [0.88118273]
 [0.88118273]
 [0.88118273]
 [0.88118273]
 [0.8821015 ]
 [0.879659  ]
 [0.879659  ]
 [0.87984616]
 [0.87978566]
 [0.87978566]
 [0.87978566]
 [0.87978566]
 [0.87978566]
 [0.8795985 ]
 [0.8795985 ]
 [0.8795985 ]
 [0.8840507 ]
 [0.8840507 ]
 [0.8840507 ]
 [0.8840507 ]
 [0.8840507 ]
 [0.8840507 ]
 [0.8840507 ]
 [0.882527  ]
 [0.8849695 ]
 [0.88003564]
 [0.88003564]
 [0.88003564]
 [0.88003564]
 [0.88003564]
 [0.88071156]
 [0.88071156]
 [0.88071156]
 [0.88071156]
 [0.88071156]
 [0.88071156]
 [0.88071156]
 [0.88071156]
 [0.88071156]
 [0.880

In [ ]:
!pip install xgboost


xgb.__version__

'0.90'

#**XGBOOST DEPLOYMENT**

In this part, the trained model will be used to predict the whole datasets

`y_pred = rgr.predict(Data that wants to be predicted)`

In the next section, uncomment and create directory if you wish to save the plots

`plt.savefig('your/directory/plot number' + str(n+1),dpi=300)`

In [ ]:
'''
y_pred = rgr.predict(X)

mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2= r2_score(y_test, y_pred)

print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )
print('MSE Value: {:.2f}'.format(mse))
print('RMSE Value: {:.2f}'.format(rmse))
print('R-squared Value: {:.2f}'.format(r2))
'''

In [ ]:
'''
plt.plot(y_test,'#0474BE', y_pred,'#D95319')
plt.legend(('Actual Data','Predicted Data'),
           loc='lower right', shadow=True )
plt.title('TestData  vs Predicted  Output')
  #plt.savefig('/content/drive/MyDrive/00 MACHINE LEARNING/y_pred (test data) ' + str(i),dpi=300)
plt.show()
'''